In [1]:
import urllib.request
import json
import os
import time

os.makedirs('result', exist_ok=True)

search_term = ""
sort_key = 'newest'
category_list = [16,331,332,333,334,335,336,337,52,362,338,51,339,340,341,342] # technology category
query_base = "https://www.kickstarter.com/projects/search.json?term=%s&category_id=%d&page=%d&sort=%s"

def get_json_data(search_term, sort_key, category_list, query_base):
    for category_id in category_list:
        for page_id in range(1,201):
            try:
                query = query_base % (search_term, category_id, page_id, sort_key)
                print(query)
                data = urllib.request.urlopen(query).read().decode("utf-8")
                response_json = json.loads(data)
            except:
                break

            # 1ページあたり、20件の結果が返ってくるので、1件ずつ保存する
            for project in response_json["projects"]:
                filepath = "result/%d.json" % project["id"]
                fp = open(filepath, "w")
                fp.write(json.dumps(project, sort_keys=True, indent=2))
                fp.close()

            # 1アクセスごとに1秒のウェイトを入れることで、過剰アクセスを防止
            time.sleep(1)
            
# get_json_data(search_term, sort_key, category_list, query_base)
# かなり時間が掛かるので動かさない時はコメントアウト

In [3]:
import glob
import json
import pandas
import pandas.io.json

# globでresultフォルダにあるファイルを走査して読み込む
project_list = [ json.loads(open(filename).read()) for filename in glob.glob("result/*.json") ]

# json_normalizeを使ってDataFrameに変換する
df = pandas.io.json.json_normalize(project_list)

# 末尾が"_at"で終わるunixtimeのカラムをdatetimeに変換する
datetime_columns = filter(lambda a: a[-3:] == "_at", df.columns)
for column in datetime_columns:
    df[column] = pandas.to_datetime(df[column], unit="s")

# DataFrameからCSV形式のstrに変換する
csv_data = df.to_csv()

# windowsのExcelに読み込ませるので、CP932にする
csv_data = csv_data.encode("cp932", "ignore")


# 結果を書き込む
fp = open("kickstarter_result.csv", "wb")
fp.write(csv_data)
fp.close()